<a href="https://colab.research.google.com/github/wfos3241/TextMining/blob/main/ex05_%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C(%ED%95%9C%EA%B8%80)_%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습 내용
- 협업필터링
- 영화 데이터 로드 및 확인
- 사용자-아이템 평점 행렬 생성
- 결측치 처리
- 영화 간 유사도 계산
- 개인별 미관람 영화에 대해 추천
- 아이템 기반 최근접 이웃 협업필터링 추천 시스템 보완
  - Threshold 필터링 추가
  - MSE/RMSE 계산

# 협업(협력) 필터링 (Collaborative Filtering)
- 참고 : https://lsjsj92.tistory.com/568


<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand03.png" width=30%>
</center>

- 취향이 비슷한 친구들에게 물어보는 것과 유사한 방식
- 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 <u><b>사용자 행동 양식</u></b>을 기반으로 추천을 수행하는 것

```python
예를 들어,
신작 영화가 나왔을 때 영화를 보러 갈지 말지를 어떻게 결정하는가?
→ "영화 관람료", "영화관 이동시간", "예고편", "전문가 평", "좋아하는 배우/감독" 등을 고려하여 영화 선택을 하게 된다면,
→ 실망한 적도 있을 거임

- 그래서 어쩌면 그 영화를 본 가까운 친구들에게 영화가 어땠는지를 물어보는게 가장 많이 애용하는 방법일 것임("단, 취향이 비슷한 친구들")
```

- 협업 필터링 (Collaborative Filtering) 종류
  - **아이템 기반 협업 필터링 (Item based Collaborative Filtering)**, **최근접이웃 협업필터링 (Nearest Neigbhor Collaborative Filtering)**
   - 사용자가 특정 아이템을 선호한다면 그와 유사한 콘텐츠를 추천해 주는 방식
   - 사용자가 콘텐츠에 부여한 평점, 구매이력 등의 데이터를 기반으로 추천해 주는 방식
   - 사용자-아이템 행렬에서 사용자가 평가하지 않는 콘텐츠를 추천해주는 것
     - **사용자 기반** : 비슷한 고객들이 ~한 아이템을 소비
     - **아이템 기반** : ~한 아이템을 소비한 고객들이 다음과 같은 아이템도 구매

 - **잠재요인 기반 협업 필터링 (Latent Factor based Collaborative Filtering)**
    - 행렬 분해를 기반으로 사용, 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재요인을 추출하는 방법
    - 아이템 기반보다는 더 활용되는 방법
    - 사용자-아이템 행렬을 사용자 잠재요인과 아이템 잠재요인으로 분해
    - 잠재요인으로 분해하면 파라미터를 감소시키는 효과

- 일반적으로 사용자 기반보다는 <u><b>아이템 기반이 정확도가 더 높음</u></b>
  - 비슷한 영화(또는 상품)을 좋아한다고 해서 사람들의 취향이 비슷하다고 판단하기 어려운 경우가 많음
```python
예를 들어,
매우 유명한 영화는 취향과 관계없이 대부분의 사람이 관람하는 경우가 많고,
사용자들이 평점을 매긴 영화(또는 상품)의 개수가 많지 않은 경우가 일반적인데,
이를 기반으로 다른 사람과의 유사도를 비교하기가 어려운 부분이 있음
```

### 데이터 로드
- 데이터셋 출처 : https://grouplens.org/datasets/movielens/latest/
- Grouplens 사이트에서 만든 MovieLens 데이터셋 활용

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/텍스트마이닝

/content/drive/MyDrive/Colab Notebooks/텍스트마이닝


In [18]:
import pandas as pd

# 영화에 대한 메타 정보(제목, 장르)를 가지고 있는 영화 정보 데이터
movies = pd.read_csv('./data/movies.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#### Movies DataFrame

| Column  | Type   | Description                                                 |
|---------|--------|-------------------------------------------------------------|
| movieId | int64  | 영화 고유 ID. 각 영화의 식별자로, 다른 데이터셋과 연결 시 사용됨                                          |
| title   | object | 영화 제목 (출시 연도 포함). 예: "Toy Story (1995)"                                   |
| genres  | object | 영화 장르 정보. 여러 장르가 "\|" 기호로 구분되어 기록됨              |

---

In [19]:
# 사용자별로 영화에 대한 평점을 매긴 데이터
ratings = pd.read_csv('./data/ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


| Column    | Type    | Description                                                           |
|-----------|---------|-----------------------------------------------------------------------|
| userId    | int64   | 사용자의 고유 ID. 평점을 남긴 사용자를 식별하는데 사용됨                                     |
| movieId   | int64   | 영화 고유 ID. movies 데이터셋의 movieId와 연결되어 영화 정보 참조에 활용됨              |
| rating    | float64 | 사용자가 영화에 부여한 평점. 일반적으로 0.5 간격의 값으로 영화 선호도를 표현                       |
| timestamp | int64   |평점이 기록된 시각. 유닉스 타임스탬프 형태로 저장되어, 평가 시점을 나타냄                              

In [20]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [21]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


- 데이터 전처리

In [22]:
# 추천에 필요없는 timestamp(평점이 기록된 시각) 제외
ratings = ratings.drop(columns = "timestamp")
ratings.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


### 사용자-아이템 평점 행렬 생성
- 협업 필터링 기반은 최근접 이웃 방식과 잠재 요인 방식으로 나뉘는데, 두 방식 모두 사용자-아이템 행렬을 사용하여 추천을 수행

- 사용자-아이템 평점 행렬
  - 각 행은 사용자를, 각 열은 아이템(영화, 상품 등)을 나타내며, 셀에는 해당 사용자가 해당 아이템에 대해 매긴 평점이나 구매, 클릭 등의 상호작용 데이터가 기록된 행렬
  - 여기서 NaN은 해당 아이템에 대한 평가가 없는 상태를 의미
  
<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand04.png" width=70%>
</center>

In [23]:
# 영화 제목 정보를 추가하기 위해 movies 데이터셋과 ratings 데이터셋을 movieId를 기준으로 병합
rating_movies = pd.merge(ratings, movies, on = 'movieId')
rating_movies.head(3)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller


In [24]:
ratings_matrix = rating_movies.pivot_table(
    index = 'userId',  # 행 : 사용자 아이디 (고유값)
    columns = 'title', # 열 : 영화 제목 (고유값)
    values = 'rating'  # 값 : 평점
)

display(ratings_matrix.head(3))
ratings_matrix.shape
# 행 : 610명의 고유 사용자 아이디
# 열 : 9719개의 고유 영화 아이디
# ex) 1번 사용자가 "¡Three Amigos! (1986)"를 보고 4.0점을 부여함 (매기지 않은 경우 NaN)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(610, 9719)

- 결측치 처리

In [25]:
# NaN 값은 해당 사용자가 해당 영화를 평가하지 않은 것으로, 0으로 채워줌
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

# 610명 사용자의 영화별 평점

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 영화와 영화간 유사도 계산

- 기억 기반 (Memorial Based) 또는 최근접 이웃 (Nearest Neighbor) 기반
  - 사용자 기반 (User Based) : 사용자 a와 비슷한 사용자 b가 이 아이템을 사용함

<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand05.png" width=60%>
</center>

  > 1. 사용자 A와 B가 영화(또는 상품)에 대해 비슷한 평점을 주었다면,  
  > 2. 사용자 B가 높게 평점을 준 영화("프로메테우스")를 사용자 A에게도 추천해주는 방식  
    - 장점 : "나와 비슷한 사람은 어떤 걸 좋아할까?"라는 관점이므로, 사용자의 실제 취향을 직접 반영  
    - 단점 : 사용자 수가 많아질수록 모든 사용자의 유사도를 계산하기가 어려움(확장성 문제)  

  - 아이템 기반 (Item Based) : A 아이템을 구매한 다른 고객은  B 아이템도 구매함

<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand06.png" width=60%>
</center>

  > 1. "프로메테우스"와 "다크나이트"가 유사한 평가 패턴을 가지고 있다면,
  > 2. 사용자가 "프로메테우스"를 좋아했다면 "다크나이트"도 추천해주는 방식
     - 장점 : 아이템(영화/상품) 간의 특성은 비교적 바뀌지 않아서, 사용자 기반보다 정확도가 높은 경우가 많음
     - 단점 : 새로운 아이템(신작 영화 등)에 대한 평가가 쌓이지 않으면, 유사도 계산이 어려움 (콜드 스타트 문제)  

- 아이템 기반을 위한 행렬 전치

In [26]:
# 아이템 기반을 위한 "행 = 영화 제목, 열 = 사용자 평점"으로 변경
# 전치(Transpose)
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 영화 간(다중, 다중)의 코사인 유사도 계산

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

# 사용자별 영화 평점을 기반으로 영화 간(다중, 다중)의 코사인 유사도 계산
# 해당 영화에 평점을 준 사용자 리스트의 유사도를 계산

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim.shape

(9719, 9719)

In [28]:
# ratings_matrix의 columns를 활용하여 데이터프레임화
ratings_matrix.columns

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9719)

In [31]:
# 영화 제목을 인덱스와 컬럼에 매핑하여 영화간 유사도 출력
item_sim_df = pd.DataFrame(item_sim, index = ratings_matrix.columns, columns = ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [37]:
# 예시: "Avengers: Infinity War - Part I (2018)"
# 유사한 영화들을 유사도 내림차순 정렬로 상위 5개 출력
# [1:6] "슬라이싱 시작값 1" : 자기 자신(가장 높은 유사도) 제외
item_sim_df["Avengers: Infinity War - Part I (2018)"].sort_values(ascending = False)[1:6]

# 만들어진 아이템 기반 유사도 데이터는 사용자의 평점 정보를 모두 취합해 영화에 따라 유사한 다른 영화를 추천할 수 있음
# 유사도 점수가 높을수록 두 영화의 평가 패턴이 비슷함을 의미

# 대부분 어벤져스와 연관된 Marvel(또는 DC) 히어로물 영화

,Avengers: Infinity War - Part I (2018)
title,
Deadpool 2 (2018),0.802636
Thor: Ragnarok (2017),0.781017
Untitled Spider-Man Reboot (2017),0.696221
Justice League (2017),0.604227
Guardians of the Galaxy 2 (2017),0.591055


| 영화 제목                           | 설명                                                                                                                                                                                   |
|-------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 어벤져스: 인피니티 워 (2018)         | Marvel Cinematic Universe (MCU)의 핵심 작품<br>타노스가 인피니티 스톤을 모아 우주 생명체의 절반을 없애려는 계획에 맞서 어벤져스와 동맹들이 싸우는 대규모 크로스오버 영화                      |
| 데드풀 2 (2018)                     | Marvel Comics의 안티히어로 데드풀이 주인공<br>웨이드 윌슨(데드풀)이 개인적 비극 후 젊은 돌연변이 러셀을 보호하려는 이야기를 다루며, 코믹하고 폭력적인 요소가 특징인 R등급 영화          |
| 토르: 라그나로크 (2017)              | MCU의 토르 시리즈 세 번째 작품<br>아스가르드의 멸망을 막기 위한 토르의 모험을 그린 영화                                                                                               |
| 스파이더맨: 홈커밍 (2017)           | Marvel Comics의 스파이더맨이 주인공<br>MCU와 Sony Pictures의 협력 작품으로 고등학생 피터 파커가 스파이더맨으로서의 정체성을 찾아가는 과정을 그린 영화                                 |
| 저스티스 리그 (2017)                | DC Comics의 슈퍼히어로 팀을 기반으로 한 영화<br>DC 확장 유니버스(DCEU)의 주요 작품이며 배트맨, 원더우먼, 아쿠아맨, 사이보그, 플래시 등이 지구를 위협하는 적에 맞서 싸우는 이야기              |
| 가디언즈 오브 갤럭시 Vol. 2 (2017)    | MCU의 가디언즈 오브 갤럭시 시리즈 두 번째 작품<br>우주를 배경으로 한 모험과 가족의 의미를 탐구하는 이야기를 담은 영화                                                                   |


> - 위 아이템 기반 영화 유사도 데이터는 `전체 사용자의 평점을 기준`으로 영화 간 유사도를 산출하여 영화 추천이 가능함  
- 하지만, 영화 간의 `단순 유사도`만 고려하기 때문에 `보았던 작품`에 대해 <u><b>중복 추천되는 한계</u></b>가 있음  

### 개인별로 아직 관람하지 않은 영화에 대해 예측 평점을 계산하고, 상위 영화를 추천

- 위의 아이템 유사도(Item Similarity)와 사용자의 기존 평점 자료를 활용한 예측 평점 로직 구현<br>
- 추천 결과를 바탕으로 Top-N 영화리스트 출력을 실습

```python
"특정한 개인"이 "아직 관람하지 않은 영화"에 대해
"아이템 유사도"와 "기존 관람한 영화의 평점데이터"를 기반으로 하여
"새롭게 모든 영화의 예측 평점"을 구한 후 높은 예측 평점을 가진 영화를 추천하는 방식
```

- 협업 필터링에서 자주 사용되는 **가중 평균** 방식 활용
  - 가중평균 : 자료의 평균을 구할 때 자료 값의 중요도나 영향 정도에 해당하는 가중치를 반영하여 구한 평균값
  - 아직 보지 않은 영화의 평점을 `비슷한 영화들의 평점`을 <U>**유사도**</U>로 <U>**가중치**</U>를 주어 <U>**평균**</U>내서 추정하는 방법 활용


> ##### 계산 순서
> 1. **분자 (Weighted Sum)** : 각 이웃 아이템의 **유사도 × 사용자가 매긴 평점**을 모두 더함
> 2. **분모 (Normalization)** : 이웃 아이템의 **유사도 절대값**을 모두 더해 정규화 기준을 만듦  
> 3. **나눗셈** : 분자 ÷ 분모 → 예측 평점 완성


> ##### 수식 :
> $\hat{R}_{u,i} = \frac{\sum_{n=1}^{N} (s_{i,n} \times r_{u,n})}{\sum_{n=1}^{N} \left| s_{i,n} \right|}$ <br>
> $\hat{R}_{u,i}$: 사용자 $u$가 아이템 $i$에 대해 예측한 평점<br>
> $s_{i,n}$: 아이템 $i$와 아이템 $n$ 간의 유사도<br>
> $r_{u,n}$: 사용자 $u$가 아이템 $n$에 실제로 매긴 평점<br>
> $N$: 아이템 $i$와 유사도가 높은 상위 $N$개 아이템의 집합<br>

- Dot Product(분자 계산, Weighted Sum)의 의미
  - 유사도×평점을 모두 더해 <U>**사용자 선호의 총합 점수**</U>를 구함
  - 유사도가 높고 평점이 높을수록, 더 큰 값으로 기여
  - 이 값은 `사용자가 아직 보지 않은 영화에 대해 사용자가 얼마나 좋아할지`를 합산한 점수
  - 이후 분모로 나누어 평균화하면, `예측 평점`이 됨

```python
# 예시 비유
- 친구 A,B 세 명의 추천력(유사도)과 만족도(평점)이 아래와 같다면:
  - 친구 A: 추천력 0.8, 만족도 4.0 → 기여도 3.2
  - 친구 B: 추천력 0.3, 만족도 2.0 → 기여도 0.6
  - 친구 C: 추천력 0.9, 만족도 5.0 → 기여도 4.5
- 기여도 총합 = 3.2 + 0.6 + 4.5 = 8.3
- 이 총합을 친구 추천력 총합(2.0)으로 나누면 4.15라는 평균 만족도를 얻음
```

- 예측 평점이 높은 Top-N 영화를 추천 리스트로 활용

In [39]:
from tqdm import tqdm
import numpy as np

def predict_ratings_item_based(user_id, ratings_matrix, item_sim_df, k_neighbors=20):
    """
    아이템 기반 가중 평균을 이용해 사용자가 보지 않은 영화의 예측 평점을 계산

    Args:
        user_id (int): 평점을 예측할 사용자 ID
        ratings_matrix (pd.DataFrame): (행:userId, 열:영화제목) 평점 행렬 (0=미관람)
        item_sim_df (pd.DataFrame): (행·열:영화제목) 영화 간 유사도 행렬
        k_neighbors (int): 예측 시 사용할 Top-k 이웃 영화 개수

    Returns:
        pd.Series: 영화 제목을 인덱스로, 예측 평점을 값으로 하는 내림차순 정렬된 시리즈
    """
    # 1) 사용자 평점 정보 로드
    user_ratings = ratings_matrix.loc[user_id]

    # 2) 미관람 영화 (user_ratings == 0, 평점이 없는) 목록 선정
    unseen_items = user_ratings[user_ratings == 0].index
    predictions = {}

    # 3) 각 미관람 영화마다 예측 평점 계산
    for movie in tqdm(unseen_items):
        # 3-1) 대상 영화와 모든 영화 간 유사도 조회
        movie_similarities = item_sim_df[movie]

        # 3-2) 사용자가 이미 본 영화 (user_ratings > 0, 평점이 있는)목록 조회
        watched = user_ratings[user_ratings > 0].index

        # 3-3) Top-k 이웃 영화 선택 (유사도 기준)
        top_neighbors = movie_similarities[watched].nlargest(k_neighbors)

        # 4) 분자 계산: dot product
        #   - ∑(유사도 × 평점) : 유사도와 평점을 모두 곱한 후 합산 (가중 합)
        #   - 각 이웃 영화의 '유사도 × 사용자의 평점'을 한 번에 계산 (dot product)
        #   - 유사도가 높고 평점이 높을수록 가중치가 커져 사용자의 선호를 잘 반영
        weighted_sum = np.dot(top_neighbors.values, user_ratings[top_neighbors.index].values)

        # 5) 분모 계산: normalization
        #    - (∑ |유사도|) : 유사도 절대값 합
        normalization = np.abs(top_neighbors).sum()

        # 6) 예측 평점 = 분자 ÷ 분모
        #   - 모든 유사도 절대값을 합산해 예측 평점의 스케일을 조정 (정규화 인자)
        #   - normalization으로 나누어 유사도 총합 차이 보정
        pred_rating = weighted_sum / normalization if normalization != 0 else 0
        predictions[movie] = pred_rating

    # 7) 결과 반환: 내림차순 정렬된 Series
    return pd.Series(predictions).sort_values(ascending=False)

In [40]:
# 예측 대상 사용자 설정
user_id = 1

# 대상 사용자가 본 영화에 대한 예측 평점 계산
pred_ratings = predict_ratings_item_based(user_id, ratings_matrix, item_sim_df, k_neighbors=20)
pred_ratings

100%|██████████| 9487/9487 [00:24<00:00, 383.13it/s]


,0
anohana: The Flower We Saw That Day - The Movie (2013),5.0
Fullmetal Alchemist: The Sacred Star of Milos (2011),5.0
Fullmetal Alchemist 2018 (2017),5.0
Blue Exorcist: The Movie (2012),5.0
Steins;Gate the Movie: The Burden of Déjà vu (2013),5.0
...,...
Annabelle: Creation (2017),0.0
Always Watching: A Marble Hornets Story (2015),0.0
My Blueberry Nights (2007),0.0
The Shallows (2016),0.0


In [41]:
pred_ratings.sort_values(ascending=False).head(20)
# 사용자 1번에게 가장 잘 맞을 것으로 예측된 영화 20개

,0
Fullmetal Alchemist: The Sacred Star of Milos (2011),5.0
Alvarez Kelly (1966),5.0
Tokyo Idols (2017),5.0
Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017),5.0
Kingsglaive: Final Fantasy XV (2016),5.0
Entertaining Angels: The Dorothy Day Story (1996),5.0
"Fireworks, Should We See It from the Side or the Bottom? (2017)",5.0
"Devil and Daniel Johnston, The (2005)",5.0
Sword Art Online The Movie: Ordinal Scale (2017),5.0
Broken English (1996),5.0


### 아이템 기반 최근접 이웃 협업 필터링 추천 시스템 보완


- Threshold 필터링 추가

In [42]:
def predict_ratings_item_based(
    user_id,
    ratings_matrix,
    item_sim_df,
    k_neighbors=20,
    similarity_threshold=0.0
):
    """
    아이템 유사도 기반 가중 평균을 이용해 사용자가 보지 않은 영화의 예측 평점을 계산
    threshold 필터링으로 동일 평점 문제를 완화

    Args:
        user_id (int): 평점을 예측할 사용자 ID
        ratings_matrix (pd.DataFrame): (행:userId, 열:영화제목) 평점 행렬 (0=미관람)
        item_sim_df (pd.DataFrame): (행·열:영화제목) 영화 간 유사도 행렬
        k_neighbors (int): 예측 시 사용할 Top-k 이웃 영화 개수
        similarity_threshold (float): 유사도 최소 임계값 (이하 필터링)

    Returns:
        pd.Series: 영화 제목을 인덱스로, 예측 평점을 값으로 하는 내림차순 정렬된 시리즈
    """
    # 1) 사용자 평점 정보 로드
    user_ratings = ratings_matrix.loc[user_id]
    # 2) 미관람 영화 목록 선정
    unseen_items = user_ratings[user_ratings == 0].index
    predictions = {}

    # 3) 각 미관람 영화마다 예측 평점 계산
    for movie in tqdm(unseen_items, desc = "Predicting ratings"):
        # 3-1) 대상 영화와 모든 영화 간 유사도 조회
        sims = item_sim_df[movie]
        # 3-2) 사용자가 이미 본 영화 목록 조회
        watched = user_ratings[user_ratings > 0].index

        ####################################################
        # 3-3) 유사도 임계값 적용
        filtered = sims[watched][sims[watched] > similarity_threshold]
        # 3-4) Top-k 이웃 영화 선택 (유사도 기준)
        if len(filtered) >= k_neighbors:
            neighbors = filtered.nlargest(k_neighbors)
        else:
            neighbors = sims[watched].nlargest(k_neighbors)
        ####################################################

        # 4) 분자 계산: dot product (Weighted Sum)
        weighted_sum = np.dot(neighbors.values, user_ratings[neighbors.index].values)
        # 5) 분모 계산: normalization (유사도 절대값 합)
        normalization = np.abs(neighbors).sum()
        # 6) 예측 평점 = 분자 ÷ 분모
        pred_rating = weighted_sum / normalization if normalization != 0 else 0
        predictions[movie] = pred_rating

    # 7) 결과 반환: 내림차순 정렬된 Series
    return pd.Series(predictions)

In [43]:
# 사용자 id
user_id = 5

# 아이템 기반 협업 필터링으로 예측 평점 계산
pred_ratings = predict_ratings_item_based(
    user_id,                   # 평점을 예측할 사용자
    ratings_matrix,            # (행:userId, 열:영화제목) 평점 행렬 (0=미관람)
    item_sim_df,               # (행·열:영화제목) 영화 간 유사도 데이터프레임
    k_neighbors = 20,          # Top‑20 이웃 영화 기준
    similarity_threshold = 0.5 # 유사도 최소 임계값(0.5 이상)
)

pred_ratings
# => Series 형태: 인덱스=영화제목, 값=예측 평점

Predicting ratings: 100%|██████████| 9675/9675 [00:24<00:00, 394.08it/s]


,0
'71 (2014),3.493327
'Hellboy': The Seeds of Creation (2004),3.767070
'Round Midnight (1986),3.593753
'Salem's Lot (2004),3.000000
'Til There Was You (1997),2.834187
...,...
eXistenZ (1999),3.681906
xXx (2002),3.742783
xXx: State of the Union (2005),3.763799
¡Three Amigos! (1986),3.542128


In [44]:
# 예측 평점 순으로 내림차순 정렬 후 상위 20개 추출
top_20 = pred_ratings.sort_values(ascending=False).head(20)
top_20
# => 사용자 5번에게 가장 잘 맞을 것으로 예측된 영화 20편과 그 평점

,0
Addams Family Reunion (1998),5.000000
Amer (2009),5.000000
Looker (1981),5.000000
Frankie and Johnny (1966),5.000000
"Harmonists, The (1997)",5.000000
This Property is Condemned (1966),5.000000
Galaxy of Terror (Quest) (1981),5.000000
Trippin' (1999),5.000000
Master of the Flying Guillotine (Du bi quan wang da po xue di zi) (1975),5.000000
Dancemaker (1998),5.000000


- MSE/RMSE 계산

In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# 0) 기존의 ratings_matrix, predict_ratings_item_based 함수, movies/ratings 데이터는 이미 로드되어 있다고 가정

# 1) 평가할 대상 사용자와 원본 평점 Series
user_id = 5
true_ratings = ratings_matrix.loc[user_id]              # 사용자 5의 실제 평점

# 2) 평점이 매겨진 영화 목록 추출
rated_items = true_ratings[true_ratings > 0].index      # 실제 평가된 영화들

# 3) train/test 분할 (예: 20%를 테스트용으로 보류)
train_items, test_items = train_test_split(
    rated_items, test_size=0.2, random_state=42
)

# 4) 학습용 평점 행렬 생성 (테스트 아이템 평점을 0으로 마스킹)
ratings_matrix_train = ratings_matrix.copy()
ratings_matrix_train.loc[user_id, test_items] = 0       # 사용자 5의 test_items 평점 제거

# 5) 아이템 유사도 재계산
ratings_matrix_T_train = ratings_matrix_train.T
item_sim_train = cosine_similarity(ratings_matrix_T_train, ratings_matrix_T_train)
item_sim_df_train = pd.DataFrame(
    item_sim_train,
    index=ratings_matrix.columns,
    columns=ratings_matrix.columns
)

# 6) 학습용 데이터로 예측 평점 계산
pred_ratings_train = predict_ratings_item_based(
    user_id,
    ratings_matrix_train,
    item_sim_df_train,
    k_neighbors=20,
    similarity_threshold=0.5
)

# 7) 테스트 아이템에 대한 실제 vs. 예측 평점 준비
y_true = true_ratings[test_items]       # 숨겨둔 실제 평점
y_pred = pred_ratings_train[test_items] # 해당 영화들에 대한 예측 평점

Predicting ratings: 100%|██████████| 9684/9684 [00:26<00:00, 367.51it/s]


In [46]:
# 8) MAE, RMSE 계산
mae  = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred) ** 0.5

# 9) 결과 출력
print(f"User {user_id} — MAE: {mae:.4f}, RMSE: {rmse:.4f}")

User 5 — MAE: 0.6372, RMSE: 0.7047


## 5. 성능 개선 방안
- **유사도(metric) 최적화**  
  - Pearson 상관계수, Cosine + Shrinkage, Jaccard 등 다양한 유사도 지표 실험  
  - 유사도 임계값(threshold)·k_neighbors 조정  
- **정규화(Normalization) & 중심화(Centering)**  
  - 사용자·아이템 평균 평점 제거 (평균 중심화)  
  - Z‑score normalization  
- **Significance Weighting**  
  - 공통 평가 개수가 적은 이웃에 대한 가중치 감소  
- **데이터 희소성 대응**  
  - SVD, PCA 등 차원 축소 기법으로 latent factor 학습  
  - ALS(Alternating Least Squares), NMF(Non‑negative Matrix Factorization)  
- **Implicit Feedback 활용**  
  - 클릭·뷰·구매 이력 가중치로 전환하여 confidence 기반 모델 적용  

## 6. 모델 기반 협업 필터링
- **행렬 분해(Matrix Factorization)**  
  - SVD, ALS, NMF → 사용자·아이템 잠재요인(latent factor) 학습  
  - 확장성 우수, 잡음 제거, 희소성 완화  
- **딥러닝 & 그래프 추천**  
  - Autoencoder, Neural CF, GNN 기반 추천  
- **장·단점**  
  - 장점: 대규모 데이터 처리, 예측 정확도 높음  
  - 단점: 학습 비용·시간 증가, 과적합 위험, 해석 어려움  

## 7. 하이브리드 추천 시스템
- **결합 방식**  
  1. 가중 결합(Weighted Hybrid)  
  2. 스위칭(Switching)  
  3. 캐스케이드(Cascade)  
  4. 특징 결합(Feature Combination)  
- **콘텐츠 기반 연계**  
  - 아이템 메타정보(장르·키워드)와 평점 유사도 결합  
  - Cold‑start 신작 영화에 콘텐츠 점수 우선 적용  
- **장·단점**  
  - 장점: 콜드 스타트·희소성·다양성 문제 동시 보완  
  - 단점: 시스템 설계·튜닝 복잡성 증가  

## 8. 추천 기법별 장·단점 비교

| 기법                    | 장점                                         | 단점                                         |
|------------------------|--------------------------------------------|---------------------------------------------|
| 사용자 기반 CF         | 개인 취향 직접 반영                          | 확장성 저하, 희소성·Cold‑start 문제           |
| 아이템 기반 CF         | 안정적·확장성 우수, 계산 비용 낮음           | 신규 아이템 처리 어려움                      |
| 모델 기반 CF           | 대규모 처리·희소성 완화, 잡음 제거           | 학습 비용·시간 증가, 해석 난이도              |
| 콘텐츠 기반 필터링      | Cold‑start 대응, 설명 가능, 구현 단순        | 심층적 사용자 취향 반영 한계                  |
| 하이브리드              | 각 기법 단점 보완, 성능·다양성 개선          | 설계·튜닝 복잡, 리소스 추가 필요             |
